In [28]:
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

In [29]:
## Load pkl file

model = load_model('model.h5')

with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('one_hot_encoder.pkl', 'rb') as f:
    one_hot_encoder = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)


In [60]:
# Example

input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',   
    'Age': 40,
    'Tenure': 3,
    'Balance': 100000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [61]:
[input_data['Geography']]

['France']

In [62]:
one_hot_encoded_data = one_hot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_data = pd.DataFrame(one_hot_encoded_data, columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_data

c:\Users\ayush_hakhu\Downloads\code\gen_ai\venv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [63]:
# This cell is now handled in the next cell

In [64]:
# Display the input data
print("Input data:")
for key, value in input_data.items():
    print(f"  {key}: {value}")

Input data:
  CreditScore: 600
  Geography: France
  Gender: Male
  Age: 40
  Tenure: 3
  Balance: 100000
  NumOfProducts: 2
  HasCrCard: 1
  IsActiveMember: 1
  EstimatedSalary: 60000


In [65]:
# Create the data in the correct order to match training data
# First, create a DataFrame with all original features
input_df = pd.DataFrame(input_data, index=[0])

# Encode gender first
input_df['Gender'] = label_encoder.transform(input_df['Gender'])

# Drop Geography column
input_df = input_df.drop(columns=['Geography'])

# Combine with one-hot encoded geography features
# The order should match the training data: original features + geography features
combined_data = pd.concat([input_df, geo_encoded_data], axis=1)

print("Combined data shape:", combined_data.shape)
print("Combined data columns:", combined_data.columns.tolist())
combined_data

Combined data shape: (1, 12)
Combined data columns: ['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_France', 'Geography_Germany', 'Geography_Spain']


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,100000,2,1,1,60000,1.0,0.0,0.0


In [66]:
combined_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,100000,2,1,1,60000,1.0,0.0,0.0


In [67]:
# scaling data
scaled_data = scaler.transform(combined_data)

print("Scaled data shape:", scaled_data.shape)
print("Scaled data:")
print(scaled_data)

Scaled data shape: (1, 12)
Scaled data:
[[-0.53598516  0.91324755  0.10479359 -0.69539349  0.38262839  0.80843615
   0.64920267  0.97481699 -0.70296551  1.00150113 -0.57946723 -0.57638802]]


In [68]:
# Make prediction
prediction = model.predict(scaled_data)
prediction_probability = prediction[0][0]

print(f"Prediction probability: {prediction_probability:.4f}")
print(f"Predicted class: {'Customer will churn' if prediction_probability > 0.5 else 'Customer will stay'}")
print(f"Confidence: {max(prediction_probability, 1-prediction_probability)*100:.2f}%")


1/1 [==============================] - 0s 394ms/step
Prediction probability: 0.0805
Predicted class: Customer will stay
Confidence: 91.95%
